In [206]:
import numpy as np
import scipy
import cvxpy as cp
import dccp

import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import EfficientSU2, TwoLocal

from qiskit.quantum_info import SparsePauliOp, Statevector, Operator

from qiskit.utils import QuantumInstance

from qiskit import Aer
from qiskit.opflow import I,X,Y,Z
from qiskit.opflow.expectations import PauliExpectation
from qiskit.opflow.converters import CircuitSampler
from qiskit.opflow.state_fns import StateFn, CircuitStateFn
from qiskit.providers.aer import AerSimulator
from qiskit.algorithms.optimizers import SPSA, SLSQP
from qiskit.algorithms import VQE, NumPyMinimumEigensolver

import itertools
import random

In [132]:
def local_op_basis(num_qubits):
    pauli_type = ['I','X','Y','Z']
    I = np.eye(2)
    X = np.array([[0,1+0.j],[1,0]])
    Y = np.array([[0,-1j],[1j,0]])
    Z = np.array([[1+0.j,0],[0,-1]])
    gate_dict = {'X': X, 'Y': Y, 'Z': Z}

    L_str_list = []
    L_mat_list = []
    L_1q_list = []

    # Single qubit op
    for n in range(num_qubits):
        for pauli in ['X','Y','Z']:
            pauli_str = 'I'*n + pauli + 'I'*(num_qubits-n-1)
            if pauli_str != 'I'*num_qubits:
                L_str_list.append(pauli_str)
                L_1q_list.append(pauli_str)

                mat = np.kron(np.kron(np.eye(2**n), gate_dict[pauli]), np.eye(2**(num_qubits-n-1)))
                L_mat_list.append(mat)
            

    # Two-local op
    for n in range(num_qubits):
        for pauli1 in ['X','Y','Z']:
            for pauli2 in ['X','Y','Z']:
                        
                gate_str = list('I'*num_qubits)
                gate_str[n] = pauli1
                gate_str[(n+1) % num_qubits] = pauli2
                L_str_list.append(''.join(gate_str))

                if n != num_qubits - 1:
                    mat = np.kron(np.kron(np.kron(np.eye(2**n), gate_dict[pauli1]), gate_dict[pauli2]), np.eye(2**(num_qubits-n-2)))
                else:
                    mat = np.kron(np.kron(gate_dict[pauli2], np.eye(2**(num_qubits - 2))), gate_dict[pauli1])
                L_mat_list.append(mat)
                
                
    L_dict = dict(zip(L_str_list, L_mat_list))
    return L_dict, L_1q_list

In [124]:
def local_op_basis_v2(num_qubits, k):
    assert num_qubits >= k
    pauli_type = ['I','X','Y','Z']
    gate_dict = {'X': X, 'Y': Y, 'Z': Z}
    L_list = []
    for n in range(num_qubits):
        all_local_ops = itertools.product(['I','X','Y','Z'], repeat=k)
        for local_op in all_local_ops:
            pauli_str = ['I'] * num_qubits
            for i,char in enumerate(local_op[::-1]):
                pauli_str[(n + i)%num_qubits] = char
            
            pauli_str = ''.join(pauli_str)
            if pauli_str != 'I'*num_qubits:
                L_list.append(pauli_str)
    return list(set(L_list))    

In [168]:
def correlation_matrix(L_list, L_mat_list, ansatz, theta):
    # Ansatz(theta) = v
    
    m = len(L_list)
    M = np.zeros((m,m), dtype='complex128')
    
    init_state = Statevector.from_label('0' * num_qubits)
    v = init_state.evolve(
            ansatz.assign_parameters(dict(zip(ansatz.parameters, theta)))).data
    
    L_mean_list = np.array([np.real(v.conj().T @ L_mat @ v) for L_mat in L_mat_list])
    
    
    for i, L_i in enumerate(L_mat_list):
        L_i_mean = L_mean_list[i]
        
        for j, L_j in enumerate(L_mat_list):
            if j >= i:
                L_j_mean = L_mean_list[j]
                anticom = L_i @ L_j + L_j @ L_i
                anticom_mean = np.real(v.conj().T @ anticom @ v)
                M[i,j] = anticom_mean/2. - L_i_mean * L_j_mean
            else:
                M[i,j] = M[j,i]    
    
    return M

In [4]:
def true_ground_state_energy(H):
#     npme = NumPyMinimumEigensolver()
#     result = npme.compute_minimum_eigenvalue(operator=Operator(H))
#     ref_value = result.eigenvalue.real
    eigvals, _ = np.linalg.eigh(H.to_matrix())
    ref_value = eigvals[0]
    return ref_value

In [71]:
def VQE_numerical_solver(H, ansatz, x0=None):
    He = H.to_matrix()

    if x0 is None:
        x0 = np.random.normal(size=ansatz.num_parameters)
        
    init_state = Statevector.from_label('0' * num_qubits)

    def func(x):
        #print(x)
        final_state = init_state.evolve(
            ansatz.assign_parameters(dict(zip(ansatz.parameters, x)))).data
        energy = np.real(final_state.conj().T @ He @ final_state)
        #print(energy)
        return energy

    #bounds = scipy.optimize.Bounds(lb=-np.pi, ub=np.pi)
    res = scipy.optimize.minimize(func, x0, method='COBYLA', tol=1e-5)
    #res = scipy.optimize.fmin_l_bfgs_b(func, x0)
    return res

In [38]:
def VQE_simulator_solver(H, ansatz, x0=None):
    seed = 70
    # define your backend or quantum instance (where you can add settings)
    backend = Aer.get_backend('aer_simulator')
    qi = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed, shots=4096)

    psi = CircuitStateFn(ansatz)
    measurable_expression = StateFn(H, is_measurement=True).compose(psi)
    expectation = PauliExpectation().convert(measurable_expression)

    def func(x):
        sampler = CircuitSampler(qi).convert(expectation, dict(zip(ansatz.parameters, x)))
        energy = sampler.eval().real
        return energy

    if x0 is None:
        x0 = np.random.normal(size=ansatz.num_parameters)
    optimizer = SPSA(maxiter=100)
    res = optimizer.minimize(fun=func, x0=x0)
    return res

In [203]:
H_op

SparsePauliOp(['IIIZI', 'IIIXX', 'IIIZZ', 'IIIZX', 'ZIIII', 'ZXIII', 'YIIIZ', 'ZYIII', 'IIXZI', 'YYIII', 'IXXII', 'IIIYI', 'ZIIIX', 'YIIIY', 'ZIIIZ', 'IIYII', 'IIZYI', 'IIIXI', 'IXIII', 'IIIXZ', 'IYIII', 'IIIIY', 'IIIZY', 'IIIIZ', 'IYZII', 'XIIIX', 'IIXII', 'IZXII', 'IYXII', 'IIYZI', 'ZIIIY', 'IIXXI', 'IIYYI', 'IYYII', 'IIYXI', 'YXIII', 'IXZII', 'XIIIY', 'IIZII', 'IIXYI', 'IIIIX', 'IIZZI', 'IXYII', 'XZIII', 'IZYII', 'IIIYY', 'ZZIII', 'YZIII', 'XXIII', 'XYIII', 'IZZII', 'XIIIZ', 'IIIXY', 'YIIIX', 'IIZXI', 'IIIYZ', 'YIIII', 'IIIYX', 'XIIII', 'IZIII'],
              coeffs=[0.00291248+0.j, 0.14073447+0.j, 0.23752612+0.j, 0.0837511 +0.j,
 0.01309604+0.j, 0.07385567+0.j, 0.10456332+0.j, 0.10935781+0.j,
 0.02321014+0.j, 0.19108168+0.j, 0.05386455+0.j, 0.2356729 +0.j,
 0.05747205+0.j, 0.09025109+0.j, 0.20401572+0.j, 0.04274788+0.j,
 0.10131156+0.j, 0.23034486+0.j, 0.07482988+0.j, 0.20286998+0.j,
 0.10188798+0.j, 0.08083658+0.j, 0.0115315 +0.j, 0.08528948+0.j,
 0.1784382 +0.j, 0.04691634+0.j, 

In [209]:
random.sample(full_L_list, 10)

['YIIIX',
 'IIIIX',
 'IIIYY',
 'YZIII',
 'ZXIII',
 'IIYYI',
 'IIZZI',
 'IIIZI',
 'IIIYI',
 'IIIXX']

In [222]:
num_qubits = 7
op_dict = {'I':I, 'X':X, 'Y':Y, 'Z':Z}
#L_dict, L_1q_list = local_op_basis(num_qubits)
full_L_list = local_op_basis_v2(num_qubits, 3)
full_L_1q_list = local_op_basis_v2(num_qubits, 1)
#full_L_op_list = [SparsePauliOp(L) for L in full_L_list]
#full_L_mat_list = [L_op.to_matrix() for L_op in full_L_op_list]


num_pauli_terms = int(len(full_L_list)/10)
L_list = random.sample(full_L_list, num_pauli_terms)
L_list.append('X'+'I'*(num_qubits-1))
L_list = list(set(L_list))
L_1q_list = [L for L in L_list if L in full_L_1q_list]
L_op_list = [SparsePauliOp(L) for L in L_list]
L_mat_list = [L_op.to_matrix() for L_op in L_op_list]


coeffs = np.random.rand(len(L_list))
norm = np.linalg.norm(coeffs)
H_coeffs = coeffs / norm

H = dict(zip(L_list, H_coeffs))
H_op = SparsePauliOp(L_list, H_coeffs)

print('Problem Ground Energy', true_ground_state_energy(H_op))

Problem Ground Energy -1.892415992109702


In [225]:
ansatz = TwoLocal(num_qubits, ['ry','rz'], 'crx', 'circular', reps=3, insert_barriers=True)
# Keep only first 1q gates at the first iteration
#start_ansatz = ansatz.bind_parameters(dict(zip(ansatz.parameters[2*num_qubits:], [0]*(ansatz.num_parameters - 2*num_qubits))))
start_ansatz = QuantumCircuit(num_qubits)
start_params = ParameterVector('θ', 2*num_qubits)

for q in range(num_qubits):
    start_ansatz.ry(start_params[q], q)
    start_ansatz.rz(start_params[q+num_qubits], q)

In [226]:
res = VQE_numerical_solver(H_op, ansatz)
print(res)

     fun: -1.5291505029687429
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 1000
  status: 2
 success: False
       x: array([ 1.88658408e+00,  1.68118425e+00,  2.23290990e+00, -2.36320872e-01,
        3.86717368e-01, -6.63506615e-01,  5.87040609e-01,  2.37346502e+00,
        6.89833991e-01,  4.88777087e-01,  9.38753009e-01,  8.12354353e-01,
        1.66804086e+00, -1.95938311e-01, -1.09287137e-01,  2.51277408e+00,
       -3.87728565e-01, -1.19246324e+00, -1.01141612e-01, -8.18488923e-01,
        1.44677283e+00, -1.69575862e-02,  7.53691468e-01, -1.79612980e+00,
        2.10101475e-01, -7.38057917e-01,  2.57405441e-04,  2.89978330e-01,
       -1.25499636e+00, -1.46309100e+00, -3.94091380e-01,  9.77145230e-02,
        3.57797261e+00, -9.82598168e-02,  2.18967845e+00,  1.50089965e+00,
        8.96700776e-02, -2.34999065e-01,  1.14183462e+00,  5.49312895e-02,
        1.72127673e+00, -4.62233716e-02, -2.97486176e-01, -1.11636258e+00,
       -

In [227]:
num_iterations = 100

## Prepare for first iteration
prev_w = np.zeros(len(L_list))
for i,L_str in enumerate(L_list):
    if L_str in L_1q_list:
        prev_w[i] = H_coeffs[i]
    else:
        prev_w[i] = 0

print(prev_w)
diff_to_H = 1
diff_to_prev = 2
    
## Iterates...

for it in range(num_iterations):
    print('Iteration: ', it+1)
    Hp = dict(zip(L_list, prev_w))
    Hp_op = SparsePauliOp(L_list, prev_w)
    
    if it == 0:
        res = VQE_numerical_solver(Hp_op, start_ansatz)
        ground_energy = res.fun
        ground_theta = np.concatenate((res.x, np.zeros(ansatz.num_parameters - len(res.x))))
    else:
        res = VQE_numerical_solver(Hp_op, ansatz, ground_theta)
        ground_energy = res.fun
        ground_theta = res.x
        
    print('True Ground Energy: ', true_ground_state_energy(Hp_op))
    print('Est Ground Energy: ', ground_energy)
    
    M = correlation_matrix(L_list, L_mat_list, ansatz, theta=ground_theta)
    
    # Construct the problem
    w = cp.Variable(len(L_list))
    
    m = 1
    delta = (1.-m*it**2)/(1.+m*it**2) + 1
    
    a = 1/diff_to_prev**2
    b = 1/diff_to_H**2
    a = a/(a+b)
    b = 1-a
    
    objective = cp.Minimize(a*cp.sum_squares(w - H_coeffs) + b*cp.sum_squares(w - prev_w))
    
    constraints = [cp.sum_squares(w) == 1, cp.sum_squares(M @ w) <= np.sqrt(diff_to_prev**2 + diff_to_H**2)]
#    constraints = [cp.sum_squares(M @ w) <= delta]
#    constraints = [cp.sum_squares(M @ w) <= delta, cp.sum_squares(w - prev_w) <= 1e-3/np.sqrt(np.log(it+2))]
    
    prob = cp.Problem(objective, constraints)

    # The optimal objective value is returned by `prob.solve()`.
    try:
        ls_res = prob.solve(method='dccp')
    except:
        print('SCS')
        ls_res = prob.solve(method='dccp', solver='SCS')
    # The optimal value for x is stored in `x.value`.
    null_deviation = np.linalg.norm(M @ w.value)
    diff_to_H = np.linalg.norm(w.value - H_coeffs)
    diff_to_prev = np.linalg.norm(w.value - prev_w)
    print('Null-space deviation:', null_deviation)
    print('Difference to H: ', diff_to_H)
    print('Difference to prev: ', diff_to_prev)
    prev_w = w.value


#     w = prev_w - 0.01 * 2 * (prev_w - H_coeffs)
#     ## Find Q = span(M)
#     Q,_ = np.linalg.qr(M)

#     ## Decompose w = Qw + Pw
#     Qw = M @ M @ w;
#     Pw = w - Qw;
#     norm_Pw = np.linalg.norm(Pw)
#     prev_w = Pw /norm_Pw

#     ## Verify Px is in nullspace of M.
#     error_MPw = np.linalg.norm(M @ prev_w)
#     print('Error: ', error_MPw)


#     def func(phi, *args):
    #     w,n,M = args
    #     var = w * np.cos(phi) + n * np.sin(phi)
    #     cost = np.linalg.norm(var - H_coeffs)**2 + 4*np.linalg.norm(M @ var)**2
    #     return cost
#     g = 2*(w - H_coeffs) + (2*4)*M @ M @ w
#     h = g - (g @ w) * w
#     n = h / np.linalg.norm(h)
#     res = scipy.optimize.minimize(func, 0, args=(w,n,M), method='L-BFGS-B', options={'maxiter': 1000})
#     phi = res.x
#     print(phi)
#     prev_w = w * np.cos(phi) + n * np.sin(phi)
    
    print('----')
    

[0.         0.         0.26495072 0.         0.         0.02544394
 0.         0.         0.19644949 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00789479 0.         0.         0.         0.
 0.         0.         0.         0.        ]
Iteration:  1
True Ground Energy:  -0.4869617477118377
Est Ground Energy:  -0.4869617477056638
Null-space deviation: 0.6426877319760594
Difference to H:  0.7000322397408092
Difference to prev:  0.7311778640131102
----
Iteration:  2
True Ground Energy:  -1.819572395078772
Est Ground Energy:  -1.7187657905069127
Null-space deviation: 0.49861505953594093
Difference to H:  0.37165922109599836
Difference to prev:  0.33968963360913695
----
Iteration:  3
True Ground Energy:  -1.9155114042718593
Est Ground Energy:  -1.77308606976601
Null-space deviation: 0.48044852186588555
Difference to H:  0.1697486435573176
Difference to prev:  0.20352619026834798


capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 

In [230]:
np.linalg.eigh(M)

(array([0.00845156, 0.02969145, 0.04329808, 0.05946863, 0.08581519,
        0.11669514, 0.12393226, 0.17481631, 0.2202163 , 0.25076341,
        0.29754882, 0.35438241, 0.47702849, 0.54123532, 0.65933012,
        0.77784242, 0.80442411, 0.85429069, 0.86227211, 0.95222074,
        0.98100087, 1.00204614, 1.01566283, 1.07575292, 1.14877638,
        1.32007017, 1.46963791, 1.56519827, 1.65581735, 1.74342626,
        1.84650411, 1.92751993, 2.07739997, 2.3419295 ]),
 array([[ 0.00587786+0.j, -0.09954871+0.j,  0.05765812+0.j, ...,
         -0.12526277+0.j,  0.22428833+0.j, -0.28119999+0.j],
        [ 0.27701139+0.j, -0.01716356+0.j,  0.10190494+0.j, ...,
         -0.06857359+0.j,  0.14850349+0.j,  0.01547673+0.j],
        [-0.02056198+0.j, -0.03300789+0.j,  0.61423006+0.j, ...,
          0.33844793+0.j, -0.05376038+0.j,  0.0380869 +0.j],
        ...,
        [ 0.00280579+0.j,  0.00914035+0.j, -0.01246054+0.j, ...,
          0.28904183+0.j, -0.21033477+0.j,  0.18758818+0.j],
        [ 0.04502